https://github.com/walsvid/CoordConv/blob/master/coordconv.py
https://medium.com/analytics-vidhya/encoding-time-series-as-images-b043becbdbf3
https://eng.uber.com/coordconv/
https://towardsdatascience.com/reading-charts-with-convolutional-neural-networks-cbaabdd5f478

# Intro

In this notebook we build a CNN classifier for the problem. The input data for this classifier consists of 1 image of the Gramian Angular Difference field for the past month, 1 image of the area plot of smoothed log prices for the past month, and a collection of numerical data on past month returns & volatility.

In [1]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from coordconv import CoordConv2d

import copy
import argparse
import os
import logging
import sys
from tqdm import tqdm
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

from ipywidgets import IntProgress
from ipywidgets import FloatProgress
from IPython.display import display


In [2]:
# Create a custom data loader for the train, test, & validation data
class NumericalAndImageDataset(Dataset):
    def __init__(self, overview_file: str, transform=None):
        """
        Initialize this dataloader
        :param overview_file: location of the overview file
        :param transform: transformer for the images
        """
        self.overview= pd.read_csv(overview_file)
        self.transform = transform


    def __len__(self):
        return len(self.overview.index)


    def __getitem__(self, idx):
        img_path_1m_bar = self.overview["1_month_img_bar"].iloc[idx]

        img_1m_bar = Image.open(img_path_1m_bar).convert('L') # Store image as greyscale (1-channel)

        num_features = torch.tensor(self.overview[[
            "1_month_return", "6_month_return", "12_month_return",
            "1_month_volatility", "6_month_volatility", "12_month_volatility",
            "stock"
        ]].iloc[idx].values).type(torch.FloatTensor)

        label = self.overview.label_1m.iloc[idx]

        if self.transform:
            img_1m_bar = self.transform(img_1m_bar)

        return img_1m_bar, num_features, label

In [3]:
class OneImageNet(nn.Module):
    def __init__(self):
        super(OneImageNet, self).__init__()

        self.image_1_features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.image_1_flat = nn.Sequential(
            nn.Dropout(p=0.25),
            nn.Linear(15 * 15 * 64, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, 3)
        )

    def forward(self, img_1, num_features):
        step1 = self.image_1_features(img_1)

        step1 = step1.view(step1.size(0), -1)

        step1 = self.image_1_flat(step1)

        return step1

In [4]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0
    running_corrects = 0
    running_total = 0

    df_collector = []

    for img_1m_bars, num_features, labels in test_loader:
        outputs = model(img_1m_bars, num_features)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * img_1m_bars.size(0)
        running_corrects += torch.sum(preds == labels.data)

        act_vs_pred_temp = pd.DataFrame({
            "actual": labels.data.numpy(),
            "pred": preds.numpy()
        })

        df_collector.append(act_vs_pred_temp)

        running_total += len(img_1m_bars)

    total_loss = running_loss / running_total
    total_acc = running_corrects.double() / running_total

    act_vs_pred = pd.concat(df_collector)

    print("Test Loss: {}".format(total_loss))
    print("Test Accuracy: {}".format(total_acc))

    return total_loss, total_acc, act_vs_pred

In [5]:
def train(model, train_loader, validation_loader, criterion, optimizer, epochs):
    datasets = {'train':train_loader, 'valid':validation_loader}
    #log = Report(epochs)

    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    for epoch in range(epochs):
        print("Epoch: " + str(epoch))

        f = IntProgress(min=0, max=17500 * 0.7) # instantiate the bar
        display(f) # display the bar

        f1 = FloatProgress(min=0, max=1)
        display(f1)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
                running_loss = 0.0
                running_corrects = 0
                running_total = 0

                for pos, (img_1m_bars, num_features, labels) in enumerate(datasets[phase]):
                    f.value += len(img_1m_bars)

                    outputs = model(img_1m_bars, num_features)
                    loss = criterion(outputs, labels)

                    if phase=='train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * img_1m_bars.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    running_total += len(img_1m_bars)

                    f1.value = running_corrects / running_total

                train_losses.append(running_loss / running_total)
                train_accs.append(running_corrects / running_total)

                print("Train Loss: {}".format(running_loss / running_total))
                print("Train Accuracy: {}".format(running_corrects / running_total))

            if phase == "valid":
                val_loss, val_acc, _ = test(model, datasets[phase], criterion)
                val_losses.append(val_loss)
                val_accs.append(val_acc)

    return model, train_losses, val_losses, train_accs, val_accs

In [6]:
def create_data_loaders(batch_size):
    train_data_path = "ModelData/obs_train.csv"
    test_data_path = "ModelData/obs_test.csv"
    val_data_path = "ModelData/obs_val.csv"

    train_transform = transforms.Compose([
        # transforms.RandomResizedCrop((224, 224)),
        # transforms.RandomHorizontalFlip(),
        # transforms.Resize((30, 30)),
        transforms.ToTensor(),
    ])

    test_transform = transforms.Compose([
        # transforms.Resize((30, 30)),
        transforms.ToTensor(),
    ])

    train_data = NumericalAndImageDataset(
        overview_file=train_data_path,
        transform=train_transform
    )
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_data = NumericalAndImageDataset(
        overview_file=test_data_path,
        transform=test_transform
    )
    test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    val_data = NumericalAndImageDataset(
        overview_file=val_data_path,
        transform=test_transform
    )
    val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

    return train_data_loader, test_data_loader, val_data_loader

In [7]:
batch_size = 1028
learning_rate = 0.1
epochs = 100

train_loader, test_loader, val_loader = create_data_loaders(batch_size)
model = OneImageNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Use adaptive momentum optimizer

print("Starting Model Training")

model, train_losses, val_losses, train_accs, val_accs = train(model, train_loader, val_loader, criterion, optimizer, epochs)

print("Train Losses:")
print(train_losses)
print("Validation Losses:")
print(val_losses)
print("Train Accuracies:")
print(train_accs)
print("Validation Accuracies:")
print(val_accs)

print("Start Model Testing")

test_loss, test_acc, act_vs_pred = test(model, test_loader, criterion)

print('saved')

Starting Model Training
Epoch: 0


IntProgress(value=0, max=12250)

FloatProgress(value=0.0, max=1.0)

Train Loss: 300.6522334907366
Train Accuracy: 0.3289855122566223
Test Loss: 1.4322569498425808
Test Accuracy: 0.3333333333333333
Epoch: 1


IntProgress(value=0, max=12250)

...

FloatProgress(value=0.0, max=1.0)

Train Loss: 1.1003018881387756
Train Accuracy: 0.3333333432674408
Test Loss: 1.0994528884524453
Test Accuracy: 0.3333333333333333
Epoch: 9


IntProgress(value=0, max=12250)

FloatProgress(value=0.0, max=1.0)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(train_accs)), train_accs)
plt.plot(range(len(val_accs)), val_accs)
plt.show()

Precision

In [ ]:
print("Precision Negative Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 0) & (act_vs_pred.actual == 0)]
    ) / len(
        act_vs_pred[(act_vs_pred.pred == 0)].index
    )
))

In [ ]:
print("Precision Neutral Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 1) & (act_vs_pred.actual == 1)]
    ) / len(
        act_vs_pred[(act_vs_pred.pred == 1)].index
    )
))

In [ ]:
print("Precision Positive Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 2) & (act_vs_pred.actual == 2)]
    ) / len(
        act_vs_pred[(act_vs_pred.pred == 2)].index
    )
))

Recall

In [ ]:
print("Recall Negative Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 0) & (act_vs_pred.actual == 0)]
    ) / len(
        act_vs_pred[(act_vs_pred.actual == 0)].index
    )
))

In [ ]:
print("Recall Neutral Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 1) & (act_vs_pred.actual == 1)]
    ) / len(
        act_vs_pred[(act_vs_pred.actual == 1)].index
    )
))

In [ ]:
print("Recall Positive Returns: {}".format(
    len(
        act_vs_pred[(act_vs_pred.pred == 2) & (act_vs_pred.actual == 2)]
    ) / len(
        act_vs_pred[(act_vs_pred.actual == 2)].index
    )
))